<img src="https://raw.githubusercontent.com/inverted-ai/invertedai/master/docs/images/banner-small.png" alt="InvertedAI" width="200"/>


In [ ]:
import sys
sys.path.insert(0, "..")

import IPython
from IPython.display import display, Image, clear_output
from ipywidgets import interact

import matplotlib.pyplot as plt
import imageio
import numpy as np
import cv2
import invertedai as iai
from IPython.utils import io
import time

In [ ]:
# API key:
iai.add_apikey("")

In [ ]:
# pick a location (4 way, signalized intgersection)
location = "iai:drake_street_and_pacific_blvd"

In [ ]:
location_info_response = iai.location_info(location=location)
rendered_static_map = location_info_response.birdview_image.decode()
scene_plotter = iai.utils.ScenePlotter(rendered_static_map,
                                       location_info_response.map_fov,
                                       (location_info_response.map_center.x, location_info_response.map_center.y),
                                       location_info_response.static_actors)

In [ ]:
collisions = []
collision_index = None
is_found_collision = False

# Simulate with `initialize`, `drive` and `light` until there are collisions.
for _ in range(20): #Attempt 20 simulations looking for a collision
    light_response = iai.light(location=location)

    response = iai.initialize(
        location=location,
        agent_count=15,
        get_birdview=True,
        traffic_light_state_history=[light_response.traffic_lights_states]
    )
    agent_attributes = response.agent_attributes
    scene_plotter.initialize_recording(response.agent_states,
                                       agent_attributes=agent_attributes,
                                       traffic_light_states=light_response.traffic_lights_states)

    agent_state_history = []
    traffic_light_state_history = []

    # 20-second scene
    for t in range(100):
        light_response = iai.light(location=location, recurrent_states=light_response.recurrent_states)
        response = iai.drive(
            location=location,
            agent_attributes=agent_attributes,
            agent_states=response.agent_states,
            recurrent_states=response.recurrent_states,
            get_birdview=False,
            traffic_lights_states=light_response.traffic_lights_states,
            get_infractions=True,
            random_seed=1
        )
        scene_plotter.record_step(response.agent_states, traffic_light_states=light_response.traffic_lights_states)
        agent_state_history.append(response.agent_states)
        traffic_light_state_history.append(light_response.traffic_lights_states)

        for agent, infractions in enumerate(response.infractions):
            if infractions.collisions: collisions.append(agent)
            if len(collisions) == 2 and not is_found_collision:
                is_found_collision = True
                collision_index = t

        print(f"Iteration number {_}")
        clear_output(wait=True)

    if is_found_collision: break

print(collisions)

In [ ]:
idx = 0
collision = collisions[idx]
blame_response = iai.blame(
    location=location,
    candidate_agents=collision,
    agent_state_history=agent_state_history[:collision_index],
    traffic_light_state_history=traffic_light_state_history[:collision_index],
    agent_attributes=agent_attributes,
    get_reasons=True,
    get_confidence_score=True,
    get_birdviews=False
)
print(blame_response.blamed_result)

In [ ]:
blame_response.reasons

In [ ]:
blame_response.confidence_score

In [ ]:
%%capture
fig, ax = plt.subplots(constrained_layout=True, figsize=(15, 15))
gif_name = 'part-iai-drive-side-road-green.gif'
scene_plotter.animate_scene(
    output_name=gif_name,
    ax=ax,
    numbers=list(collision[0]),
    direction_vec=False,
    velocity_vec=False,
    plot_frame_number=True
)

In [ ]:
Image(gif_name)